<a href="https://colab.research.google.com/github/vohoaidanh/AIGCDetectBenchmark/blob/main/intrinsic_inference.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Make sure to set the runtime to GPU: Runtime -> Change runtime type -> T4 GPU

You can upload your own images, then change the relevant code cells to load it and send it through the model.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# install the intrinsic decomposition repo from github
!pip install https://github.com/compphoto/Intrinsic/archive/main.zip

In [ ]:
import torch

# import some helper functions from chrislib (will be installed by the intrinsic repo)
from chrislib.general import show, view, uninvert
from chrislib.data_util import load_image

# import model loading and running the pipeline
from intrinsic.pipeline import run_pipeline
from intrinsic.model_util import load_models

In [ ]:
# download the pretrained weights and return the model (may take a bit to download weights)
intrinsic_model = load_models('paper_weights')

In [ ]:
# load an example image from the github repo
torch.hub.download_url_to_file('https://raw.githubusercontent.com/compphoto/Intrinsic/main/figures/avocado.png', 'avo.png')

In [ ]:
# load the image to run through the pipeline
img = load_image('/content/avo.png')

In [ ]:
# run the image through the pipeline (use R0 resizing dicussed in the paper)
result = run_pipeline(
    intrinsic_model,
    img,
    resize_conf=0.0,
    maintain_size=True,
    linear=False,
    device='cuda'
)

In [ ]:
# convert the inverse shading to regular shading for visualization
shd = uninvert(result['inv_shading'])
alb = result['albedo']

In [ ]:
# show the result (gamma corrects the linear intrinsic components and scales to [0-1])
show([img, view(shd), view(alb)], size=(20, 7))

## Intrinsic Process

In [ ]:
# Copy Data from gdrive to colab
!cp source -d dest


In [ ]:
import os

def get_image_list(data_path):
  # Get all images in dataset (return a list of path of each image)
  items = os.listdir(data_path) # Each item contains 40 images
  image_list = []
  for item in items:
    f = os.path.join(data_path, item)
    imgs = os.listdir(f)
    imgs = [os.path.join(f,imgs) for i in imgs]
    image_list.extend(imgs)
  return image_list


# Run process all images in the list

In [ ]:
data_path = ''
images = get_image_list(data_path)

print('Total: {0} images'.format(len(images)))

for image in images:
    result = run_pipeline(
      intrinsic_model,
      img,
      resize_conf=0.0,
      maintain_size=True,
      linear=False,
      device='cuda'
    )

    albedo = results['albedo']
    inv_shd = results['inv_shading']

    # compute shading from inverse shading
    shading = uninvert(inv_shd)